### Differentiation on Linear Subspaces of G

In [1]:
from kingdon import Algebra
from gc_utils import *
import numpy as np
alg = Algebra(1,3)
locals().update(alg.blades)

In [2]:
#2.1
A = create_random_multivector(alg)
B = create_random_multivector(alg)
F = lambda X: (X+B)**3
X = create_random_multivector(alg)
h = 6e-6 # leave h as a global var, so to modify with ease
# central scheme, modify h to minimize truncation/round-off error
def differential(F, X, A):
    d = (h)*A
    return (1/(2*h))*F(X+d) - (1/(2*h))*F(X-d)

In [3]:
# a surgestion of h
def auto_step_size(x):
    """
    Automatically determine step size based on variable magnitude and machine epsilon.
    """
    eps = np.finfo(float).eps
    return np.maximum(np.abs(x) * np.cbrt(eps), 100 * eps)
X = create_random_multivector(alg)
np.mean(auto_step_size(X[:]))

2.660616079162052e-06

In [4]:
#2.3
A2 = create_r_blade(2, alg)
F = lambda X: P(X, A2)**2
max_diff(differential(F, X, A), differential(F, P(X,A2), A))

1.1641532182693481e-10

In [5]:
#2.9
h = 6e-6
G = lambda X: (X+A2)**3
GF = lambda X: G(F(X))
max_diff(differential(GF, X, A), differential(G, F(X), differential(F, X, A)))

1.3969838619232178e-09

In [6]:
#2.12
h = 6e-6
B = create_random_multivector(alg)
FA = lambda X: differential(F, X, A)
FB = lambda X: differential(F, X, B)
FAB = differential(FA, X, B)
FBA = differential(FB, X, A)
max_diff(FAB, FBA)

2.9103830456733704e-11

In [7]:
def derivative(F, X, alg, grade=None, frame=None, r_frame=None):
    if not frame:
        if grade or (grade == 0):
            frame = r_vector_frame(alg.frame, grade)
            r_frame = r_vector_frame(reciprocal(alg.frame), grade, reverse=True)
        else:
            frame = multi_frame(alg.frame)
            r_frame = reci_frame(alg.frame)
    dF = 0
    for v, r in zip(frame, r_frame):
        dF += r * (differential(F, X, v))
    return dF

In [8]:
#2.13
# Randomly generate several frames to see dF is independent of frame
h = 6e-6
r = 4
dFs = []
for _ in range(5):
    basis = create_r_vectors(r, alg)
    r_basis = reciprocal(basis)
    frame = multi_frame(basis)
    r_frame = multi_frame(r_basis, reverse=True)

    dF = derivative(F, X, alg, frame=frame, r_frame=r_frame)
    dFs.append(dF)
[max_diff(dFs[-1], dF) for dF in dFs[:-1]]

[4.903959460733631e-09,
 1.4873895448630492e-09,
 6.026652253510621e-09,
 1.8873576035360884e-09]

In [9]:
# For F defined on A2, we can restrict our bases
# choose a nonorthonormal-to-A2-vector e.g. e1
v1 = P(e1, A2)
# contract the parallel part
v2 = v1 | A2
frame = multi_frame([v1, v2])
r_frame = reci_frame([v1, v2])
dF = derivative(F, X, alg, frame=frame, r_frame=r_frame)
dFs[-1], dF

(4.7 + 4.12 𝐞₁ + 2.43 𝐞₂ + 2.84 𝐞₃ + 5.24 𝐞₄ + -0.708 𝐞₁₂ + 1.12 𝐞₁₃ + 0.176 𝐞₁₄ + 1.15 𝐞₂₃ + 1.01 𝐞₂₄ + -1.31 𝐞₃₄ + 7.81e-10 𝐞₁₂₃ + 4.06e-10 𝐞₁₂₄ + 1.01e-09 𝐞₁₃₄ + 1.15e-09 𝐞₂₃₄ + 7.38e-10 𝐞₁₂₃₄,
 4.7 + 4.12 𝐞₁ + 2.43 𝐞₂ + 2.84 𝐞₃ + 5.24 𝐞₄ + -0.708 𝐞₁₂ + 1.12 𝐞₁₃ + 0.176 𝐞₁₄ + 1.15 𝐞₂₃ + 1.01 𝐞₂₄ + -1.31 𝐞₃₄ + 3.08e-11 𝐞₁₂₃ + -3.28e-11 𝐞₁₂₄ + 2.99e-11 𝐞₁₃₄ + 4.22e-11 𝐞₂₃₄ + -2.22e-11 𝐞₁₂₃₄)

In [10]:
#2.18
# double differentials lower accuracy
# wise to avoid/replace in computation
# but if we just want to see the equivalence, we can just choose a fairly large h
h = 1e-2
F_ = lambda A: differential(F, X, A)
d_F_ = derivative(F_, A, alg, frame=frame, r_frame=r_frame)
# dF = derivative(F , X, alg, frame=frame, r_frame=r_frame)
max_diff(d_F_, dF), d_F_, dF

(1.277511429975675e-10,
 4.7 + 4.12 𝐞₁ + 2.43 𝐞₂ + 2.84 𝐞₃ + 5.24 𝐞₄ + -0.708 𝐞₁₂ + 1.12 𝐞₁₃ + 0.176 𝐞₁₄ + 1.15 𝐞₂₃ + 1.01 𝐞₂₄ + -1.31 𝐞₃₄ + -1.8e-12 𝐞₁₂₃ + -1.62e-14 𝐞₁₂₄ + 2.65e-12 𝐞₁₃₄ + -3.62e-13 𝐞₂₃₄ + 2.41e-13 𝐞₁₂₃₄,
 4.7 + 4.12 𝐞₁ + 2.43 𝐞₂ + 2.84 𝐞₃ + 5.24 𝐞₄ + -0.708 𝐞₁₂ + 1.12 𝐞₁₃ + 0.176 𝐞₁₄ + 1.15 𝐞₂₃ + 1.01 𝐞₂₄ + -1.31 𝐞₃₄ + 3.08e-11 𝐞₁₂₃ + -3.28e-11 𝐞₁₂₄ + 2.99e-11 𝐞₁₃₄ + 4.22e-11 𝐞₂₃₄ + -2.22e-11 𝐞₁₂₃₄)

In [11]:
# 2.19
h = 1e-5
def adjoint1(F, X, A, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = multi_frame(alg.frame)
        r_frame = reci_frame(alg.frame)
    _F = 0    
    for v, r in zip(frame, r_frame):
        _F += r * alg.sp(differential(F, X, v), A)
    return _F

# either works, but this one's slightly better
def adjoint(F, X, A, alg: Algebra, frame=None, r_frame=None):
    _F = derivative(lambda X: alg.sp(F(X), (A)), X, alg, frame=frame, r_frame=r_frame)
    return _F

_F = adjoint(F, X, A, alg)
_F1 = adjoint1(F, X, A, alg)
F_BA = differential(F, X, B).sp(A)

(
max_diff(B.sp(_F), F_BA),
max_diff(B.sp(_F1), F_BA)
)

(2.4472646131812326e-12, 2.1677382111562338e-11)

In [12]:
#2.20 F defined on A2
max_diff(P(_F, A2), _F)

4.751354865106805e-11

In [13]:
#2.21
[max_diff(_F.grade(r), derivative(lambda X: F(X).sp(A), X, alg, r)) for r in range(0, 5)]

[0.0, 0.0, 0.0, 0.0, 0.0]

In [14]:
#2.24
r = 2
_sum = derivative(lambda X: F(X), X, alg) + derivative(lambda X: G(X), X, alg)
max_diff(derivative(lambda X: F(X) + G(X), X, alg), _sum)

1.0047429555015697e-10

In [15]:
#2.25a
_prod = derivative(lambda X: F(X), X, alg)*G(X) + derivative(lambda Y:F(X)*G(Y), X, alg)
max_diff(derivative(lambda X: F(X)*G(X), X, alg), _prod)

1.8986412442245637e-09

In [16]:
#2.25b The note by Hitzer regard this as a definition. But we shall see it's not! The operators have their meaning.
# r-derivative
grade = 2
frame = r_vector_frame(alg.frame, grade)
r_frame = r_vector_frame(reciprocal(alg.frame), grade, reverse=True)
FdG = 0
Fd = 0
dG = 0
for v, r in zip(frame, r_frame):
    FdG += differential(lambda X: F(X)*r*G(X), X, v) # the over dots made clear using lambda function
    Fd += differential(F, X, v) * r
    dG += r * differential(G, X, v)
max_diff(FdG, (Fd * G(X)) + (F(X)*dG))

2.0958736968168523e-09

In [17]:
#2.25b derivative on entire frame
frame = multi_frame(alg.frame)
r_frame = reci_frame(alg.frame)
FdG = 0
Fd = 0
dG = 0
for v, r in zip(frame, r_frame):
    FdG += differential(lambda X: F(X)*r*G(X), X, v)
    Fd += differential(F, X, v) * r
    dG += r * differential(G, X, v)
max_diff(FdG, (Fd * G(X)) + (F(X)*dG))

2.679399813132477e-09

In [18]:
#2.26 r-derivative
h = 1e-4
r = 2
max_diff(
derivative(GF, X, alg, r), 
derivative(lambda Y: differential(G, F(X), F(Y)), X, alg, r)
)

1.5842387046660633e-07

In [19]:
#2.26 derivative
h = 1e-4
r = None
max_diff(
derivative(GF, X, alg, r), 
derivative(lambda Y: differential(G, F(X), F(Y)), X, alg, r)
)

4.131284576658345e-07

In [20]:
#2.28a
(
    max_diff(P(A, A2), differential(lambda X: P(X, A2), X, A)),
    max_diff(P(A, A2), derivative(lambda X: P(X, A2).sp(A), X, alg))
)

(1.1696199564426024e-12, 1.5733525593475406e-12)

In [21]:
#2.28b
max_diff(P(A, A2).reverse(), differential(lambda X: P(X.reverse(), A2), X, A))

1.1696199564426024e-12

In [22]:
#2.29 d of n dim spade is 2^n
h = 1e-5
A3 = create_r_blade(3, alg)
(
derivative(lambda X: X, X, alg), #4d
derivative(lambda X: P(X, A3), X, alg), #3d
derivative(lambda X: P(X, A2), X, alg) #2d
)

(16.0,
 8.0 + -1.71e-11 𝐞₁₂ + -5e-12 𝐞₁₃ + 7.96e-12 𝐞₁₄ + 3.62e-11 𝐞₂₃ + 3.87e-12 𝐞₂₄ + 2.89e-11 𝐞₃₄,
 4.0 + 3.18e-12 𝐞₁₂ + 1.36e-12 𝐞₁₃ + 2.18e-11 𝐞₁₄ + -1.86e-11 𝐞₂₄ + 1.05e-11 𝐞₃₄ + 2.05e-11 𝐞₁₂₃₄)

In [23]:
#2.30
max_diff(derivative(lambda X: X.sp(X.reverse()), X, alg), 2*X.reverse())

2.425393219596117e-11

In [24]:
#2.31
k=6
max_diff(differential(lambda X: X**k, X, A), sum((X**l)*A*(X**(k-1-l)) for l in range(k)))

5.9189545709159574e-08

In [25]:
#2.32 To allow different signature, we consider norm square instead
def normsq(X):
    return X.sp(X.reverse())[0]

k=3    
max_diff(derivative(lambda X: normsq(X) ** (k), X, alg), 2*k*normsq(X)**(k-1)*X.reverse())

2.2007284883329703e-09

In [26]:
#2.33
max_diff(derivative(lambda X: np.log(np.abs(normsq(X))), X, alg), 2 * X.reverse()/normsq(X))

3.572830920006709e-11

In [27]:
#2.30 var
for n in range(7):
    alg = Algebra(n)
    X = create_random_multivector(alg)
    dXXr = derivative(lambda X: X * X.reverse(), X, alg)
    Xr = X.reverse()
    ratio = np.round(dXXr[0]/Xr[0])
    print(ratio, max_diff(dXXr, ratio*Xr)) 

2.0 7.613687458274399e-12
4.0 5.835776306639673e-12
6.0 7.424283410273347e-12
8.0 4.019717891878827e-11
12.0 1.0641310055348185e-10
24.0 2.128093257169894e-10
56.0 4.411475629240158e-10


Compare with 2.2.30. The derivative is also an integer multiple of $\tilde{X}$, yet a complex one:
$$ \partial  X\tilde{X} = \.{\partial} \.{X} \tilde{X} + \.{\partial} X \.{\tilde{X}} = 2^n \tilde{X} + {\gamma}_n \tilde{X}$$
where ${\gamma}_n$ are suggests to be [1, 2, 2, 0, -4, -8, -8] for $0\leq n \leq 6$ from the above and below numerical tests:

In [28]:
for n in range(1,7):
    alg = Algebra(n)
    X = create_random_multivector(alg)
    dXXr = derivative(lambda Y: X*Y.reverse(), X, alg)
    print(n, dXXr[0]/X[0])

1 2.000000000002711
2 1.9999999999983011
3 4.066728756853823e-11
4 -4.000000000042734
5 -8.000000000040236
6 -7.9999999995977396


$\partial X^2$ is conversely neither a multiple of $X$ nor $\tilde{X}$.
But if we separate scalars or pseudoscalars from other grades, another integer multiple emerges. 
Try proving it!

In [29]:
# 2.30 var2
for n in range(2,7):
    alg = Algebra(n)
    X = create_random_multivector(alg)
    dXX = derivative(lambda X: X**2, X, alg)
    ratios = np.round(dXX[0]/X[0])
    ratiov = np.round(dXX[1]/X[1])
    ratiops = np.round(dXX[2**n-1]/X[2**n-1])
    print(ratios, ratiov, ratiops, max_diff(dXX[1:2**n-1], ratiov*(X[1:2**n-1]))) 

8.0 4.0 4.0 4.0177194904345015e-11
16.0 8.0 16.0 5.050271312256882e-11
32.0 16.0 16.0 6.496136961686716e-11
64.0 32.0 64.0 1.915765324156382e-10
128.0 64.0 64.0 5.060059038441977e-10


In [30]:
#2.37a
from scipy.special import comb
for n, r in [(5,2), (6,2), (1,1), (1,0), (2,0)]:
    alg = Algebra(n)
    X = create_random_multivector(alg)
    print(derivative(lambda X: X, X, alg, r), comb(n, r, exact=True))

10.0 10
15.0 15
1.0 1
1.0 1
1.0 1


In [31]:
#2.38a
def gamma(n, r, s):
    gamma = 0
    K = int(np.round((r + s - abs(r - s))/2))
    for k in range(0, K+1):
        gamma += (-1)**(r*s-k)*comb(r, k, exact=True)*comb(n-r, s-k, exact=True)
    return gamma

for n, r, s in [(5,2,1), (6,2,3), (1,1,1), (1,0,1), (2,0,1)]:
    alg = Algebra(n)
    X = create_random_multivector(alg)
    Ar = create_random_multivector(alg).grade(r)
    print(max_diff(derivative(lambda X: Ar*X, X, alg, s), gamma(n, r, s)*Ar)) 

2.3103519097844583e-11
4.824363131206155e-11
2.138178523125589e-12
8.29780688604842e-13
1.1541878564003127e-11


In [32]:
#2.38b
n = 5
s = 2
alg = Algebra(n)
X = create_random_multivector(alg)
A = create_random_multivector(alg)
max_diff(derivative(lambda X: A*X, X, alg, s), sum(gamma(n, r, s)*A.grade(r) for r in range(0, n+1)))


6.159317500475936e-11

Construct $\gamma_n$ from $\Gamma_s^r$

In [33]:
#2.38a reverse version
def gamma_rev(n, r, s):
    gamma = 0
    K = int(np.round((r + s - abs(r - s))/2))
    for k in range(0, K+1):
        gamma += (-1)**(r*s-k+s*(s-1)/2)*comb(r, k, exact=True)*comb(n-r, s-k, exact=True)
    return gamma

for n, r, s in [(5,2,1), (6,2,3), (1,1,1), (1,0,1), (2,0,1)]:
    alg = Algebra(n)
    X = create_random_multivector(alg)
    Ar = create_random_multivector(alg).grade(r)
    dAXr = derivative(lambda X: Ar*X.reverse(), X, alg, s)
    print(max_diff(dAXr, gamma_rev(n, r, s)*Ar)) 

4.413747145548541e-11
1.517017622632011e-10
2.509104035652854e-13
5.188072194073357e-13
1.1950440637065185e-12


In [34]:
for n in range(7):
    r_parts = np.zeros(n+1)
    for s in range(0, n+1):
        for r in range(0, n+1):
            r_parts[r] += (-1)**(r*(r-1)/2)*gamma_rev(n, r, s) # one more reversion
    print(r_parts)

[1.]
[2. 2.]
[2. 2. 2.]
[0. 0. 0. 0.]
[-4. -4. -4. -4. -4.]
[-8. -8. -8. -8. -8. -8.]
[-8. -8. -8. -8. -8. -8. -8.]


The coefficients of each grade match due to the reversion-symmetry of gamma_rev.
Thus we get $\gamma_n$ by fixing any r:

In [35]:
def gamma_n(n):
    return sum(gamma_rev(n, 0, s) for s in range(0, n+1))
[gamma_n(n) for n in range(7)]

[1.0, 2.0, 2.0, 0.0, -4.0, -8.0, -8.0]

In [36]:
#3.1a dAG and ddGA are equivalent according to 2.18
h = 1e-3
alg = Algebra(2)
A = create_random_multivector(alg)
B = create_random_multivector(alg)
G = lambda X: (X+B) ** 2

dAG = derivative(lambda A: G(A^B), A, alg)
ddGA = derivative(lambda A_: differential(G, A^B, A_^B), A, alg)

max_diff(dAG, ddGA), dAG, ddGA

(1.0162537478208833e-10,
 1.25 + 0.73 𝐞₁ + 0.31 𝐞₂ + 0.137 𝐞₁₂,
 1.25 + 0.73 𝐞₁ + 0.31 𝐞₂ + 0.137 𝐞₁₂)

In [37]:
#3.1b a headache...
h = 1e-2 # sufficiently large to see the equivalence

ddGAB = derivative(lambda B: derivative(lambda A: G(A^B), A, alg), B, alg)
dAdBG = (
    derivative(lambda B_: derivative(lambda A_: differential(G, A^B, A_^B_), A, alg), B, alg) +
    derivative(lambda B_: differential(lambda X: derivative(lambda A_: differential(G, X, A_^B), A, alg), A^B, A^B_), B, alg)
    )
max_diff(ddGAB, dAdBG), ddGAB, dAdBG

(4.119454999340633e-09,
 5.79 + 7.84 𝐞₁ + 2.31 𝐞₂ + 0.858 𝐞₁₂,
 5.79 + 7.84 𝐞₁ + 2.31 𝐞₂ + 0.858 𝐞₁₂)

In [38]:
#3.2b
h = 1e-5
for r in range(1, 7):
    alg = Algebra(r)
    G = lambda X: (X.grade(r)+B) ** 2
    frame = r_vector_frame(alg.frame, r)
    r_frame = r_vector_frame(reciprocal(alg.frame), r, reverse=True)
    for s in range(r):
        A = create_random_multivector(alg).grade(s)
        B = create_random_multivector(alg).grade(r-s)

        dAG = derivative(lambda A: G(A^B), A, alg, grade=s)
        BdG = 0
        for v, vr in zip(frame, r_frame):
            BdG += inner(B, vr) * (differential(G, A^B, v))
        assert_diff(dAG, BdG, 1e-8)
        

In [39]:
#3.2c Somehow the pseudoscalars are more ill conditioned, let me know if you figure out
h = 1e-2
for r in range(2,5):
    alg = Algebra(r)
    G = lambda X: (X.grade(r)+B) ** 2
    frame = r_vector_frame(alg.frame, r)
    r_frame = r_vector_frame(reciprocal(alg.frame), r, reverse=True)
    for s in range(r):
        A = create_random_multivector(alg).grade(s)
        B = create_random_multivector(alg).grade(r-s)

        dAG = lambda B: derivative(lambda A: G(A^B), A, alg, grade=s)
        dBdAG = derivative(dAG, B, alg, grade=r-s)
        BdG = 0
        for v, vr in zip(frame, r_frame):
            for u, ur in zip(frame, r_frame):
                BdG += inner(A, u) * inner(B, vr) * (differential(lambda X: differential(G, X, ur), A^B, v))
        BdG *= (-1)**(s*(r-s))
        BdG += comb(r, r-s)*derivative(G, A^B, alg, grade=r)

        print(dBdAG, BdG, sep="\n")

0.853 𝐞₁₂
0.603 𝐞₁₂
1.89 + 0.277 𝐞₁₂
1.89 + 0.277 𝐞₁₂
2.58 𝐞₁₂₃
1.94 𝐞₁₂₃
-3.55e-13 + -2.57 𝐞₁ + -0.373 𝐞₂ + 1.79 𝐞₃ + 4.23 𝐞₁₂ + 5.16 𝐞₁₃ + 3.69 𝐞₂₃ + 3.14 𝐞₁₂₃
3.8e-13 + -2.57 𝐞₁ + -0.373 𝐞₂ + 1.79 𝐞₃ + 4.23 𝐞₁₂ + 5.16 𝐞₁₃ + 3.69 𝐞₂₃ + 3.14 𝐞₁₂₃
5.29 𝐞₁ + 2.43 𝐞₂ + 1.13 𝐞₃ + -0.31 𝐞₁₂₃
5.29 𝐞₁ + 2.43 𝐞₂ + 1.13 𝐞₃ + -0.31 𝐞₁₂₃
0.674 𝐞₁₂₃₄
0.479 𝐞₁₂₃₄
1.08 𝐞₁₂ + 0.137 𝐞₁₃ + -0.793 𝐞₁₄ + 0.833 𝐞₂₃ + 1.24 𝐞₂₄ + 0.771 𝐞₃₄ + 5.4 𝐞₁₂₃₄
1.08 𝐞₁₂ + 0.137 𝐞₁₃ + -0.793 𝐞₁₄ + 0.833 𝐞₂₃ + 1.24 𝐞₂₄ + 0.771 𝐞₃₄ + 5.4 𝐞₁₂₃₄
-3.66 + 5.72 𝐞₁₂ + 12.6 𝐞₁₃ + 4.62 𝐞₁₄ + 12.4 𝐞₂₃ + 9.04 𝐞₂₄ + 4.47 𝐞₃₄ + 2.12 𝐞₁₂₃₄
-3.66 + 5.72 𝐞₁₂ + 12.6 𝐞₁₃ + 4.62 𝐞₁₄ + 12.4 𝐞₂₃ + 9.04 𝐞₂₄ + 4.47 𝐞₃₄ + 2.12 𝐞₁₂₃₄
0.217 𝐞₁₂ + -0.224 𝐞₁₃ + -1.11 𝐞₁₄ + 0.194 𝐞₂₃ + -0.984 𝐞₂₄ + 2.01 𝐞₃₄ + -0.71 𝐞₁₂₃₄
0.217 𝐞₁₂ + -0.224 𝐞₁₃ + -1.11 𝐞₁₄ + 0.194 𝐞₂₃ + -0.984 𝐞₂₄ + 2.01 𝐞₃₄ + -0.71 𝐞₁₂₃₄


In [40]:
#3.4
h = 1e-3
alg = Algebra(r)
C = create_random_multivector(alg)
D = create_random_multivector(alg)
L = lambda X: C * X * D
dAL = lambda B: derivative(lambda A: L(A^B), A, alg, grade=s)
dBdAL = derivative(dAL, B, alg, grade=r-s)
max_diff(dBdAL, comb(r, r-s) * derivative(L, A^B, alg, grade=r))

9.523404287392623e-11

In [41]:
def multi_differential(F, X, vectors):
    from functools import reduce
    
    def apply_differential(current_func, vector):
        return lambda X: differential(current_func, X, vector)
    
    # Reduce applies the function cumulatively to the items of iterable, from left to right,
    # so as to reduce the iterable to a single value.
    return reduce(apply_differential, vectors, F)(X)

In [42]:
def multi_derivative(F, vectors):
    def apply_derivative(current_func, V):
        assert_blade(V)
        return lambda X: derivative(lambda Y: current_func(Y ^ X), V, alg, grade=V.grades[0])
    
    return reduce(apply_derivative, vectors, F)(1)

# Excercise: use frame and differential to split the derivatives. Iterate over multiindexes sounds a good idea

In [43]:
#3.5
from math import factorial
h = 1e-2
alg = Algebra(1,3)
C = create_random_multivector(alg)
D = create_random_multivector(alg)
L = lambda X: C * X * D
r = 3
n = len(alg.frame)
frame = alg.frame
r_frame = reciprocal(alg.frame)
vectors = create_r_vectors(r, alg)
X = wedge(vectors)

drL = multi_derivative(L, vectors) 
drL, factorial(r)*derivative(L, X, alg, grade=r)


(31.3 + 39.9 𝐞₁ + 20.7 𝐞₂ + 6.56 𝐞₃ + 8.6 𝐞₄ + -3.76 𝐞₁₂ + -7.2 𝐞₁₃ + 0.0823 𝐞₁₄ + -3.28 𝐞₂₃ + -7.71 𝐞₂₄ + 1.04 𝐞₃₄ + 25.9 𝐞₁₂₃ + -13.4 𝐞₁₂₄ + 20.7 𝐞₁₃₄ + 11.0 𝐞₂₃₄ + -7.19 𝐞₁₂₃₄,
 31.3 + 39.9 𝐞₁ + 20.7 𝐞₂ + 6.56 𝐞₃ + 8.6 𝐞₄ + -3.76 𝐞₁₂ + -7.2 𝐞₁₃ + 0.0823 𝐞₁₄ + -3.28 𝐞₂₃ + -7.71 𝐞₂₄ + 1.04 𝐞₃₄ + 25.9 𝐞₁₂₃ + -13.4 𝐞₁₂₄ + 20.7 𝐞₁₃₄ + 11.0 𝐞₂₃₄ + -7.19 𝐞₁₂₃₄)

In [44]:
# We can also expand the vector derivatives. And a linear function equals to its differential
drL = 0
r = 3
for v, vr in zip(frame, r_frame):
    for u, ur in zip(frame, r_frame):
        for w, wr in zip(frame, r_frame):
            drL += vr * u * w * L(wr^ur^v)
drL

31.3 + 39.9 𝐞₁ + 20.7 𝐞₂ + 6.56 𝐞₃ + 8.6 𝐞₄ + -3.76 𝐞₁₂ + -7.2 𝐞₁₃ + 0.0823 𝐞₁₄ + -3.28 𝐞₂₃ + -7.71 𝐞₂₄ + 1.04 𝐞₃₄ + 25.9 𝐞₁₂₃ + -13.4 𝐞₁₂₄ + 20.7 𝐞₁₃₄ + 11.0 𝐞₂₃₄ + -7.19 𝐞₁₂₃₄

In [45]:
# A cleaner version of above
from itertools import permutations
drL = 0
for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
    drL += wedge(base_vectors[::-1]) * L(wedge(reci_vectors))
drL

31.3 + 39.9 𝐞₁ + 20.7 𝐞₂ + 6.56 𝐞₃ + 8.6 𝐞₄ + -3.76 𝐞₁₂ + -7.2 𝐞₁₃ + 0.0823 𝐞₁₄ + -3.28 𝐞₂₃ + -7.71 𝐞₂₄ + 1.04 𝐞₃₄ + 25.9 𝐞₁₂₃ + -13.4 𝐞₁₂₄ + 20.7 𝐞₁₃₄ + 11.0 𝐞₂₃₄ + -7.19 𝐞₁₂₃₄

In [46]:
from itertools import product
def vectors_partial(F, vectors, directions):
    r = len(vectors)
    drF = 0
    for offset in product([1,-1], repeat=r):
        offset = np.array(offset)
        coef = np.prod(offset)
        offset = offset * h
        drF += 1/(2*h)**r * coef * F([a+v*d for a, v, d in zip(vectors, directions, offset)])
    return drF

In [47]:
#3.6 simplicial derivative
from itertools import permutations
h = 1e-2
def simplicial_derivative(F, vectors, alg):
    frame = alg.frame
    r_frame = reciprocal(alg.frame)
    drF = 0
    r = len(vectors)
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        drF += wedge(base_vectors[::-1]) * vectors_partial(F, vectors, reci_vectors)
    return (1/factorial(r)) * drF

simplicial_derivative(lambda vectors: L(wedge(vectors)), vectors, alg) * factorial(r)

31.3 + 39.9 𝐞₁ + 20.7 𝐞₂ + 6.56 𝐞₃ + 8.6 𝐞₄ + -3.76 𝐞₁₂ + -7.2 𝐞₁₃ + 0.0823 𝐞₁₄ + -3.28 𝐞₂₃ + -7.71 𝐞₂₄ + 1.04 𝐞₃₄ + 25.9 𝐞₁₂₃ + -13.4 𝐞₁₂₄ + 20.7 𝐞₁₃₄ + 11.0 𝐞₂₃₄ + -7.19 𝐞₁₂₃₄

In [48]:
#3.7a
h = 1e-2
for n, r in [(5,5),(5,4),(5,3),(6,5),(6,4),(6,3)]:
    alg = Algebra(n)
    vectors = create_r_vectors(r, alg)
    print(simplicial_derivative(lambda vectors: wedge(vectors), vectors, alg))

1.0
5.0 + -3.03e-13 𝐞₁₂ + 3.03e-12 𝐞₁₃ + 6.06e-13 𝐞₁₅
10.0 + 9.47e-14 𝐞₁₂ + 9.47e-14 𝐞₁₅ + 3.79e-14 𝐞₂₃ + -7.58e-14 𝐞₂₅ + -6.44e-13 𝐞₃₅ + -6.82e-13 𝐞₄₅
6.0 + 1.86e-10 𝐞₁₂ + 7.37e-11 𝐞₁₃ + 6.21e-11 𝐞₁₄ + 4.97e-10 𝐞₁₅ + -1.09e-10 𝐞₁₆ + 1.94e-11 𝐞₂₃ + 6.21e-11 𝐞₂₄ + -6.21e-11 𝐞₂₅ + -1.55e-10 𝐞₂₆ + 6.98e-11 𝐞₃₄ + -1.05e-10 𝐞₃₅ + 3.88e-12 𝐞₃₆ + 1.24e-10 𝐞₄₅ + 3.73e-10 𝐞₄₆ + 7.76e-11 𝐞₅₆
15.0 + 2.12e-12 𝐞₁₂ + 2.09e-11 𝐞₁₃ + -1.24e-11 𝐞₁₄ + 2.97e-11 𝐞₁₅ + -7.58e-12 𝐞₁₆ + 1.82e-11 𝐞₂₃ + -1.39e-11 𝐞₂₄ + 2.76e-11 𝐞₂₅ + -7.28e-12 𝐞₂₆ + -1.82e-12 𝐞₃₄ + 3.03e-13 𝐞₃₅ + 9.09e-13 𝐞₄₅ + -3.03e-13 𝐞₅₆
20.0 + -7.58e-14 𝐞₁₃ + -4.55e-13 𝐞₁₄ + -7.58e-14 𝐞₁₅ + -2.27e-13 𝐞₁₆ + -4.55e-13 𝐞₂₄ + -7.58e-14 𝐞₂₅ + -1.52e-13 𝐞₂₆ + -7.58e-14 𝐞₃₅ + 3.79e-13 𝐞₄₅ + -1.52e-13 𝐞₅₆


In [49]:
#3.7b X^2 is certainly not a linear function. I gave the r-derivatives of X^2 in 2.30 var2
h = 1e-3
for n, r in [(5,5),(5,4),(5,3),(6,5),(6,4),(6,3)]:
    alg = Algebra(n)
    vectors = create_r_vectors(r, alg)
    print(max_diff(simplicial_derivative(lambda vectors: wedge(vectors)**2, vectors, alg), (r+1) * wedge(vectors)))

3.60267527416136e-06
1.901555181094139e-07
1.3088608641353972e-08
1.7900150726946062e-06
1.4460285779893667e-07
8.432433862282096e-09


In [50]:
#3.8 Just a ratio of factorials
r, s = 7, 4
factorial(r+s) / (factorial(r) * factorial(s)), comb(r+s, s)

(330.0, 330.0)

In [51]:
#3.9
n = 5
r = 4
h = 1e-5
alg = Algebra(n)
A_list = [create_random_multivector(alg) for _ in range(r)]
B_list = [create_random_multivector(alg) for _ in range(r)]
vectors = create_r_vectors(r, alg)
Fr = lambda vectors: sum(A_list[i] * vectors[i] * B_list[i] for i in range(r))

akpkFp = 0
akpkFn = 0
for i in range(r):
    akpkFp += Fr(vectors[:i] +[vectors[i]+vectors[i]*h] + vectors[i+1:])
    akpkFn += Fr(vectors[:i] +[vectors[i]-vectors[i]*h] + vectors[i+1:])

akpkFp *= 1/(2*h)
akpkFn *= 1/(2*h)
akpkF = akpkFp - akpkFn
akpkF, Fr(vectors)


(-3.24 + -0.216 𝐞₁ + 1.82 𝐞₂ + 3.15 𝐞₃ + -3.22 𝐞₄ + -2.27 𝐞₅ + 1.1 𝐞₁₂ + 2.44 𝐞₁₃ + -3.45 𝐞₁₄ + 3.08 𝐞₁₅ + 8.73 𝐞₂₃ + 10.7 𝐞₂₄ + 3.95 𝐞₂₅ + 6.98 𝐞₃₄ + 0.178 𝐞₃₅ + -6.88 𝐞₄₅ + 1.78 𝐞₁₂₃ + 11.1 𝐞₁₂₄ + 0.499 𝐞₁₂₅ + 5.88 𝐞₁₃₄ + -0.814 𝐞₁₃₅ + -5.64 𝐞₁₄₅ + 10.0 𝐞₂₃₄ + 9.49 𝐞₂₃₅ + 7.53 𝐞₂₄₅ + 5.48 𝐞₃₄₅ + 7.48 𝐞₁₂₃₄ + 2.06 𝐞₁₂₃₅ + 8.77 𝐞₁₂₄₅ + 6.17 𝐞₁₃₄₅ + 11.6 𝐞₂₃₄₅ + 12.1 𝐞₁₂₃₄₅,
 -3.24 + -0.216 𝐞₁ + 1.82 𝐞₂ + 3.15 𝐞₃ + -3.22 𝐞₄ + -2.27 𝐞₅ + 1.1 𝐞₁₂ + 2.44 𝐞₁₃ + -3.45 𝐞₁₄ + 3.08 𝐞₁₅ + 8.73 𝐞₂₃ + 10.7 𝐞₂₄ + 3.95 𝐞₂₅ + 6.98 𝐞₃₄ + 0.178 𝐞₃₅ + -6.88 𝐞₄₅ + 1.78 𝐞₁₂₃ + 11.1 𝐞₁₂₄ + 0.499 𝐞₁₂₅ + 5.88 𝐞₁₃₄ + -0.814 𝐞₁₃₅ + -5.64 𝐞₁₄₅ + 10.0 𝐞₂₃₄ + 9.49 𝐞₂₃₅ + 7.53 𝐞₂₄₅ + 5.48 𝐞₃₄₅ + 7.48 𝐞₁₂₃₄ + 2.06 𝐞₁₂₃₅ + 8.77 𝐞₁₂₄₅ + 6.17 𝐞₁₃₄₅ + 11.6 𝐞₂₃₄₅ + 12.1 𝐞₁₂₃₄₅)

In [52]:
#3.10
h = 1e-2

def skew_symmetrizer(F, vectors, alg):
    frame = alg.frame
    r_frame = reciprocal(alg.frame)
    drF = 0
    r = len(vectors)
    Ar = wedge(vectors)
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        drF += (Ar| wedge(base_vectors[::-1])) * vectors_partial(F, vectors, reci_vectors)
    return (1/factorial(r)) * drF

skew_symmetrizer(Fr, vectors, alg)

2.47e-10 + -1.12e-11 𝐞₁ + 1.85e-10 𝐞₂ + -6.39e-11 𝐞₃ + -3.27e-11 𝐞₄ + 9.11e-11 𝐞₅ + -9.76e-12 𝐞₁₂ + 2.29e-10 𝐞₁₃ + 5.78e-10 𝐞₁₄ + -2.9e-10 𝐞₁₅ + 3.53e-10 𝐞₂₃ + -2.55e-10 𝐞₂₄ + -4.24e-10 𝐞₂₅ + 3.82e-10 𝐞₃₄ + -9.14e-12 𝐞₃₅ + 2.16e-10 𝐞₄₅ + -1.6e-10 𝐞₁₂₃ + 6.24e-10 𝐞₁₂₄ + 3.77e-11 𝐞₁₂₅ + -3.69e-10 𝐞₁₃₄ + -5.86e-11 𝐞₁₃₅ + -8.34e-11 𝐞₁₄₅ + 5.86e-10 𝐞₂₃₄ + 4.62e-10 𝐞₂₃₅ + 1.42e-11 𝐞₂₄₅ + 4.3e-10 𝐞₃₄₅ + 9.3e-10 𝐞₁₂₃₄ + -1.91e-10 𝐞₁₂₃₅ + 6.74e-10 𝐞₁₂₄₅ + 3.58e-11 𝐞₁₃₄₅ + -3.69e-10 𝐞₂₃₄₅ + -7.46e-11 𝐞₁₂₃₄₅

In [53]:
#3.11
h = 1e-2
# skew_symmetrizer(lambda vectors: skew_symmetrizer(Fr, vectors, alg), vectors, alg)
C = create_random_multivector(alg)
D = create_random_multivector(alg)
L = lambda vectors: C * wedge(vectors) * D
skew_symmetrizer(L, vectors, alg), L(vectors)

(0.257 + 0.449 𝐞₁ + -0.351 𝐞₂ + 0.292 𝐞₃ + 0.082 𝐞₄ + 0.102 𝐞₅ + -0.511 𝐞₁₂ + -0.0928 𝐞₁₃ + -0.00308 𝐞₁₄ + 0.597 𝐞₁₅ + -1.01 𝐞₂₃ + -0.805 𝐞₂₄ + 0.115 𝐞₂₅ + 0.748 𝐞₃₄ + -0.0687 𝐞₃₅ + 0.0688 𝐞₄₅ + -0.674 𝐞₁₂₃ + -1.55 𝐞₁₂₄ + 0.279 𝐞₁₂₅ + 0.406 𝐞₁₃₄ + -0.275 𝐞₁₃₅ + -0.415 𝐞₁₄₅ + 0.739 𝐞₂₃₄ + -1.13 𝐞₂₃₅ + -0.703 𝐞₂₄₅ + 0.229 𝐞₃₄₅ + 0.523 𝐞₁₂₃₄ + -0.647 𝐞₁₂₃₅ + -1.22 𝐞₁₂₄₅ + -0.0458 𝐞₁₃₄₅ + 0.67 𝐞₂₃₄₅ + 0.415 𝐞₁₂₃₄₅,
 0.257 + 0.449 𝐞₁ + -0.351 𝐞₂ + 0.292 𝐞₃ + 0.082 𝐞₄ + 0.102 𝐞₅ + -0.511 𝐞₁₂ + -0.0928 𝐞₁₃ + -0.00308 𝐞₁₄ + 0.597 𝐞₁₅ + -1.01 𝐞₂₃ + -0.805 𝐞₂₄ + 0.115 𝐞₂₅ + 0.748 𝐞₃₄ + -0.0687 𝐞₃₅ + 0.0688 𝐞₄₅ + -0.674 𝐞₁₂₃ + -1.55 𝐞₁₂₄ + 0.279 𝐞₁₂₅ + 0.406 𝐞₁₃₄ + -0.275 𝐞₁₃₅ + -0.415 𝐞₁₄₅ + 0.739 𝐞₂₃₄ + -1.13 𝐞₂₃₅ + -0.703 𝐞₂₄₅ + 0.229 𝐞₃₄₅ + 0.523 𝐞₁₂₃₄ + -0.647 𝐞₁₂₃₅ + -1.22 𝐞₁₂₄₅ + -0.0458 𝐞₁₃₄₅ + 0.67 𝐞₂₃₄₅ + 0.415 𝐞₁₂₃₄₅)

In [54]:
def blade_split(Ar, alg):
    r = Ar.grades[0]
    projects = [P(e, Ar) for e in alg.frame[:r]]
    wed = wedge(projects)
    ratio = Ar[:][0]/wed[:][0]
    return projects, ratio

In [56]:
#3.12
r = 3
alg = Algebra(5)
locals().update(alg.blades)
Ar = create_r_blade(r, alg)
alpha = lambda vectors: wedge(vectors) | Ar
vectors, ratio = blade_split(Ar, alg)
vectors[0] *= ratio
max_diff(simplicial_derivative(alpha, vectors, alg), Ar)

4.2931491694986335e-12